In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = 15,9
rcParams['figure.dpi'] = 300
sns.set_style("darkgrid", {"axes.facecolor": ".9"})


def get_timings(filename):

    file = open("../results/" + filename + "/" + filename + "_sequential.txt", 'r')

    lines = file.readlines()

    splits = lines[0].split()

    t_seq_read = int(splits[3])

    total_time = 0
    iters = 0

    for i in range(1, len(lines)):

        total_time += int(lines[i].split()[4])
        iters += 1

    # print(f"Mean sequential time: {total_time/iters}")
    t_seq = total_time / iters

    file.close()

    file = open("../results/" + filename + "/" + filename + "_thread.txt", 'r')

    lines = file.readlines()

    splits = lines[0].split()

    t_thread_read = int(splits[4])

    workers_thread = []
    total_times_thread = []
    iters_thread = []

    last_worker = 1

    total_time = 0
    iter = 0

    for i in range(1, len(lines)):

        if last_worker != int(lines[i].split()[1][:-1]):

            # print(f"Mean sequential time for {last_worker} workers: {total_time/iter}") 
            workers_thread.append(last_worker)
            total_times_thread.append(total_time/iter)
            iters_thread.append(iter)

            last_worker = int(lines[i].split()[1][:-1])
            total_time = int(lines[i].split()[5])
            iter = 1
            
        else:

            total_time += int(lines[i].split()[5])
            iter += 1

    file.close()

    file = open("../results/" + filename + "/" + filename + "_ff.txt", 'r')

    lines = file.readlines()

    splits = lines[0].split()

    t_ff_read = int(splits[3])

    workers_ff = []
    total_times_ff = []
    iters_ff = []

    last_worker = 1

    total_time = 0
    iter = 0

    for i in range(1, len(lines)):

        if last_worker != int(lines[i].split()[1][:-1]):

            # print(f"Mean sequential time for {last_worker} workers: {total_time/iter}") 
            workers_ff.append(last_worker)
            total_times_ff.append(total_time/iter)
            iters_ff.append(iter)

            last_worker = int(lines[i].split()[1][:-1])
            total_time = int(lines[i].split()[5])
            iter = 1
            
        else:

            total_time += int(lines[i].split()[5])
            iter += 1

    return (t_seq_read, t_seq, workers_thread, workers_ff, total_times_thread, total_times_ff)




In [ ]:
filename = input("Graph size: ")

(t_seq_read, t_seq, workers_thread, workers_ff, total_times_thread, total_times_ff) = get_timings(filename)

t_seq

In [ ]:
for i in range(0, len(total_times_ff)):
    print(f"Workers: {i+1} time {total_times_ff[i]}")

In [ ]:
for i in range(0, len(total_times_thread)):
    print(f"Workers: {i+1} time {total_times_thread[i]}")

In [ ]:
filename = input("Graph size: ")

(t_seq_read, t_seq, workers_thread, workers_ff, total_times_thread, total_times_ff) = get_timings(filename)

speedup_thread = []
speedup_ff = []

for val in total_times_thread:
    speedup_thread.append(float(t_seq / val))
    
for val in total_times_ff:
    speedup_ff.append(float(t_seq / val))

tag_thread = ["Pool"] * len(speedup_thread)
tag_ff = ["FF"] * len(speedup_ff)

scalability_thread = []
scalability_ff = []

for val in total_times_thread:
    scalability_thread.append(float(total_times_thread[0] / val))
    
for val in total_times_ff:
    scalability_ff.append(float(total_times_ff[0] / val))

efficiency_thread = []
efficiency_ff = []

for speedup, num_w in zip(speedup_thread, workers_thread):
    efficiency_thread.append(float(speedup / num_w))
    
for speedup, num_w in zip(speedup_ff, workers_ff):
    efficiency_ff.append(float(speedup / num_w))


total_speedup = speedup_thread + speedup_ff 
total_scalability = scalability_thread + scalability_ff 
total_efficiency = efficiency_thread + efficiency_ff  

total_workers = workers_thread + workers_ff

tags = tag_thread + tag_ff


df = pd.DataFrame(np.column_stack([total_speedup, total_workers, tags]), 
                               columns=['SpeedUp', 'Workers', 'Tags'])

df["SpeedUp"] = pd.to_numeric(df["SpeedUp"], downcast="float")
df["Workers"] = pd.to_numeric(df["Workers"], downcast="integer")


df2 = pd.DataFrame(np.column_stack([total_scalability, total_workers, tags]), 
                               columns=['Scalability', 'Workers', 'Tags'])

df2["Scalability"] = pd.to_numeric(df2["Scalability"], downcast="float")
df2["Workers"] = pd.to_numeric(df2["Workers"], downcast="integer")


df3 = pd.DataFrame(np.column_stack([total_efficiency, total_workers, tags]), 
                               columns=['Efficiency', 'Workers', 'Tags'])

df3["Efficiency"] = pd.to_numeric(df3["Efficiency"], downcast="float")
df3["Workers"] = pd.to_numeric(df3["Workers"], downcast="integer")



sns.lineplot(
    data=df, 
    x="Workers", y="SpeedUp", hue="Tags"
)

plt.show()

sns.lineplot(
    data=df2, 
    x="Workers", y="Scalability", hue="Tags"
)

plt.show()

sns.lineplot(
    data=df3, 
    x="Workers", y="Efficiency", hue="Tags"
)

plt.show()



In [ ]:
display(df)